In [1]:
# Read in data
df = spark.read.format('csv') \
  .option("inferSchema", 'true') \
  .option("header", 'true') \
  .option("sep", ",") \
  .load("/FileStore/tables/Use_Case_6__India_s_Audt-77a21.csv")


# Exploratory Data Analysis

Let's run some basic tests to see what kind of data we're dealing with.

In [3]:
# What kind of data are we working with?
df.printSchema()

root
-- Sector_score: double (nullable = true)
-- LOCATION_ID: string (nullable = true)
-- PARA_A: double (nullable = true)
-- Score_A: double (nullable = true)
-- Risk_A: double (nullable = true)
-- PARA_B: double (nullable = true)
-- Score_B6: double (nullable = true)
-- Risk_B: double (nullable = true)
-- TOTAL: double (nullable = true)
-- numbers: double (nullable = true)
-- Score_B10: double (nullable = true)
-- Risk_C: double (nullable = true)
-- Money_Value: double (nullable = true)
-- Score_MV: double (nullable = true)
-- Risk_D: double (nullable = true)
-- District_Loss: integer (nullable = true)
-- PROB16: double (nullable = true)
-- RiSk_E: double (nullable = true)
-- History: integer (nullable = true)
-- Prob19: double (nullable = true)
-- Risk_F: double (nullable = true)
-- Score: double (nullable = true)
-- Inherent_Risk: double (nullable = true)
-- CONTROL_RISK: double (nullable = true)
-- Detection_Risk: double (nullable = true)
-- Audit_Risk: double (nullable = true)
-- Risk: integer (nullable = true)

In [4]:
df = df.drop("TOTAL", "LOCATION_ID", "Audit_Risk", "Detection_Risk")

In [5]:
# Shape of Dataframe
print("There are {} rows and {} columns.".format(df.count(), len(df.columns)))

There are 776 rows and 23 columns.

In [6]:
df = df.dropna()

In [7]:
# Let's take a look at each feature's correlational value in relation to class
new_df = df.toPandas()
corr_matrix= new_df.corr()
abs(corr_matrix["Risk"].sort_values(ascending=True))

Out[ 13 ]: 
Sector_score 0.393322
PROB16 0.176818
Risk_F 0.214423
History 0.239354
Risk_D 0.254253
Risk_B 0.255181
PARA_B 0.256920
Money_Value 0.256992
Prob19 0.298520
numbers 0.308017
Risk_C 0.342006
Score_B10 0.353664
Inherent_Risk 0.356881
PARA_A 0.378547
Risk_A 0.384869
District_Loss 0.403591
RiSk_E 0.411594
CONTROL_RISK 0.416285
Score_A 0.619383
Score_B6 0.635524
Score_MV 0.688207
Score 0.785813
Risk 1.000000
Name: Risk, dtype: float64

In [8]:
df.createOrReplaceTempView("fraud")

In [9]:
%sql

select Risk, count(0) from fraud group by Risk

Risk,count(0)
1,305
0,470


In [10]:
df.columns

Out[ 15 ]: 
['Sector_score',
 'PARA_A',
 'Score_A',
 'Risk_A',
 'PARA_B',
 'Score_B6',
 'Risk_B',
 'numbers',
 'Score_B10',
 'Risk_C',
 'Money_Value',
 'Score_MV',
 'Risk_D',
 'District_Loss',
 'PROB16',
 'RiSk_E',
 'History',
 'Prob19',
 'Risk_F',
 'Score',
 'Inherent_Risk',
 'CONTROL_RISK',
 'Risk']

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
new_df = df.withColumnRenamed("Risk","label")

features = ['Sector_score',
 'PARA_A',
 'Score_A',
 'Risk_A',
 'PARA_B',
 'Score_B6',
 'Risk_B',
 'numbers',
 'Score_B10',
 'Risk_C',
 'Money_Value',
 'Score_MV',
 'Risk_D',
 'District_Loss',
 'PROB16',
 'RiSk_E',
 'History',
 'Prob19',
 'Risk_F',
 'Score',
 'Inherent_Risk',
 'CONTROL_RISK']

assembler = VectorAssembler(
    inputCols= features,
    outputCol= "features")
output = assembler.transform(new_df)
display(output.select("features", "label").head(10))

features,label
"List(1, 22, List(), List(3.89, 4.18, 0.6, 2.508, 2.5, 0.2, 0.5, 5.0, 0.2, 1.0, 3.38, 0.2, 0.676, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 2.4, 8.574, 0.4))",1
"List(1, 22, List(), List(3.89, 0.0, 0.2, 0.0, 4.83, 0.2, 0.966, 5.0, 0.2, 1.0, 0.94, 0.2, 0.188, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 2.0, 2.554, 0.4))",0
"List(1, 22, List(), List(3.89, 0.51, 0.2, 0.102, 0.23, 0.2, 0.046, 5.0, 0.2, 1.0, 0.0, 0.2, 0.0, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 2.0, 1.548, 0.4))",0
"List(1, 22, List(), List(3.89, 0.0, 0.2, 0.0, 10.8, 0.6, 6.48, 6.0, 0.6, 3.6, 11.75, 0.6, 7.05, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 4.4, 17.53, 0.4))",1
"List(1, 22, List(), List(3.89, 0.0, 0.2, 0.0, 0.08, 0.2, 0.016, 5.0, 0.2, 1.0, 0.0, 0.2, 0.0, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 2.0, 1.416, 0.4))",0
"List(1, 22, List(), List(3.89, 0.0, 0.2, 0.0, 0.83, 0.2, 0.166, 5.0, 0.2, 1.0, 2.95, 0.2, 0.59, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 2.0, 2.156, 0.4))",0
"List(1, 22, List(), List(3.89, 1.1, 0.4, 0.44, 7.41, 0.4, 2.964, 5.0, 0.2, 1.0, 44.95, 0.6, 26.97, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 3.2, 31.774, 0.4))",1
"List(1, 22, List(), List(3.89, 8.5, 0.6, 5.1, 12.03, 0.6, 7.218, 5.5, 0.4, 2.2, 7.79, 0.4, 3.116, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 4.2, 18.034, 0.4))",1
"List(1, 22, List(), List(3.89, 8.4, 0.6, 5.04, 11.05, 0.6, 6.63, 5.5, 0.4, 2.2, 7.34, 0.4, 2.936, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 4.2, 17.206, 0.4))",1
"List(1, 22, List(), List(3.89, 3.98, 0.6, 2.388, 0.99, 0.2, 0.198, 5.0, 0.2, 1.0, 1.93, 0.2, 0.386, 2.0, 0.2, 0.4, 0.0, 0.2, 0.0, 2.4, 4.372, 0.4))",0


In [12]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(output)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(output)
scaledData.show()


+------------+------+-------+------+------+--------+------+-------+---------+------+-----------+--------+------+-------------+------+------+-------+------+------+-----+-------------+------------+-----+--------------------+--------------------+
Sector_score|PARA_A|Score_A|Risk_A|PARA_B|Score_B6|Risk_B|numbers|Score_B10|Risk_C|Money_Value|Score_MV|Risk_D|District_Loss|PROB16|RiSk_E|History|Prob19|Risk_F|Score|Inherent_Risk|CONTROL_RISK|label| features| scaledFeatures|
+------------+------+-------+------+------+--------+------+-------+---------+------+-----------+--------+------+-------------+------+------+-------+------+------+-----+-------------+------------+-----+--------------------+--------------------+
 3.89| 4.18| 0.6| 2.508| 2.5| 0.2| 0.5| 5.0| 0.2| 1.0| 3.38| 0.2| 0.676| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.4| 8.574| 0.4| 1|[3.89,4.18,0.6,2....|[0.16007297027769...|
 3.89| 0.0| 0.2| 0.0| 4.83| 0.2| 0.966| 5.0| 0.2| 1.0| 0.94| 0.2| 0.188| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.0| 2.554| 0.4| 0|[3.89,0.0,0.2,0.0...|[0.16007297027769...|
 3.89| 0.51| 0.2| 0.102| 0.23| 0.2| 0.046| 5.0| 0.2| 1.0| 0.0| 0.2| 0.0| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.0| 1.548| 0.4| 0|[3.89,0.51,0.2,0....|[0.16007297027769...|
 3.89| 0.0| 0.2| 0.0| 10.8| 0.6| 6.48| 6.0| 0.6| 3.6| 11.75| 0.6| 7.05| 2| 0.2| 0.4| 0| 0.2| 0.0| 4.4| 17.53| 0.4| 1|[3.89,0.0,0.2,0.0...|[0.16007297027769...|
 3.89| 0.0| 0.2| 0.0| 0.08| 0.2| 0.016| 5.0| 0.2| 1.0| 0.0| 0.2| 0.0| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.0| 1.416| 0.4| 0|[3.89,0.0,0.2,0.0...|[0.16007297027769...|
 3.89| 0.0| 0.2| 0.0| 0.83| 0.2| 0.166| 5.0| 0.2| 1.0| 2.95| 0.2| 0.59| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.0| 2.156| 0.4| 0|[3.89,0.0,0.2,0.0...|[0.16007297027769...|
 3.89| 1.1| 0.4| 0.44| 7.41| 0.4| 2.964| 5.0| 0.2| 1.0| 44.95| 0.6| 26.97| 2| 0.2| 0.4| 0| 0.2| 0.0| 3.2| 31.774| 0.4| 1|[3.89,1.1,0.4,0.4...|[0.16007297027769...|
 3.89| 8.5| 0.6| 5.1| 12.03| 0.6| 7.218| 5.5| 0.4| 2.2| 7.79| 0.4| 3.116| 2| 0.2| 0.4| 0| 0.2| 0.0| 4.2| 18.034| 0.4| 1|[3.89,8.5,0.6,5.1...|[0.16007297027769...|
 3.89| 8.4| 0.6| 5.04| 11.05| 0.6| 6.63| 5.5| 0.4| 2.2| 7.34| 0.4| 2.936| 2| 0.2| 0.4| 0| 0.2| 0.0| 4.2| 17.206| 0.4| 1|[3.89,8.4,0.6,5.0...|[0.16007297027769...|
 3.89| 3.98| 0.6| 2.388| 0.99| 0.2| 0.198| 5.0| 0.2| 1.0| 1.93| 0.2| 0.386| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.4| 4.372| 0.4| 0|[3.89,3.98,0.6,2....|[0.16007297027769...|
 3.89| 5.43| 0.6| 3.258| 10.77| 0.6| 6.462| 5.0| 0.2| 1.0| 4.42| 0.2| 0.884| 2| 0.2| 0.4| 0| 0.2| 0.0| 3.6| 12.004| 0.4| 1|[3.89,5.43,0.6,3....|[0.16007297027769...|
 3.89| 15.38| 0.6| 9.228| 40.14| 0.6|24.084| 5.0| 0.2| 1.0| 0.96| 0.2| 0.192| 2| 0.4| 0.8| 1| 0.4| 0.4| 4.0| 35.704| 1.2| 1|[3.89,15.38,0.6,9...|[0.16007297027769...|
 3.89| 5.47| 0.6| 3.282| 7.63| 0.4| 3.052| 5.0| 0.2| 1.0| 10.43| 0.6| 6.258| 2| 0.2| 0.4| 1| 0.4| 0.4| 3.6| 14.392| 0.8| 1|[3.89,5.47,0.6,3....|[0.16007297027769...|
 3.89| 1.09| 0.4| 0.436| 0.35| 0.2| 0.07| 5.0| 0.2| 1.0| 0.0| 0.2| 0.0| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.2| 1.906| 0.4| 0|[3.89,1.09,0.4,0....|[0.16007297027769...|
 3.89| 0.0| 0.2| 0.0| 0.84| 0.2| 0.168| 5.0| 0.2| 1.0| 0.007| 0.2|0.0014| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.0| 1.5694| 0.4| 0|[3.89,0.0,0.2,0.0...|[0.16007297027769...|
 3.89| 1.95| 0.4| 0.78| 9.01| 0.4| 3.604| 5.0| 0.2| 1.0| 9.0| 0.4| 3.6| 2| 0.2| 0.4| 0| 0.2| 0.0| 3.0| 9.384| 0.4| 1|[3.89,1.95,0.4,0....|[0.16007297027769...|
 3.89| 8.54| 0.6| 5.124| 31.63| 0.6|18.978| 5.0| 0.2| 1.0| 41.28| 0.6|24.768| 2| 0.2| 0.4| 1| 0.4| 0.4| 4.2| 50.67| 0.8| 1|[3.89,8.54,0.6,5....|[0.16007297027769...|
 3.89| 4.18| 0.6| 2.508| 4.83| 0.2| 0.966| 5.5| 0.4| 2.2| 14.03| 0.6| 8.418| 2| 0.2| 0.4| 0| 0.2| 0.0| 3.2| 14.492| 0.4| 1|[3.89,4.18,0.6,2....|[0.16007297027769...|
 3.89| 1.81| 0.4| 0.724| 1.03| 0.2| 0.206| 5.0| 0.2| 1.0| 0.0| 0.2| 0.0| 2| 0.2| 0.4| 0| 0.2| 0.0| 2.2| 2.33| 0.4| 0|[3.89,1.81,0.4,0....|[0.16007297027769...|
 3.89| 4.86| 0.6| 2.916| 46.78| 0.6|28.068| 5.5| 0.4| 2.2| 63.18| 0.6|37.908| 2| 0.2| 0.4| 0| 0.2| 0.0| 4.4| 71.492| 0.4| 1|[3.89,4.86,0.6,2....|[0.16007297027769...|
+------------+------+-------+

In [13]:
(train, test) = scaledData.randomSplit([0.8, 0.2], seed=0)

# Cache the training and test datasets
train.cache()
test.cache()

Out[ 18 ]: DataFrame[Sector_score: double, PARA_A: double, Score_A: double, Risk_A: double, PARA_B: double, Score_B6: double, Risk_B: double, numbers: double, Score_B10: double, Risk_C: double, Money_Value: double, Score_MV: double, Risk_D: double, District_Loss: int, PROB16: double, RiSk_E: double, History: int, Prob19: double, Risk_F: double, Score: double, Inherent_Risk: double, CONTROL_RISK: double, label: int, features: vector, scaledFeatures: vector]

In [14]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(labelCol='label', featuresCol = 'scaledFeatures')
evaluator = BinaryClassificationEvaluator()

# Create the pipeline
pipeline = Pipeline(stages=[dtc])

# Cross Validate the data and get the best model
paramGrid = ParamGridBuilder().addGrid(dtc.maxBins, [65, 65, 65]).addGrid(dtc.maxDepth, [4, 6, 8]).build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

# Use the best model for predictions
cv_model = crossval.fit(train)
bestModel = cv_model.bestModel
predictions = bestModel.transform(test)
error= evaluator.evaluate(predictions)
print(str(error))


0.9591836734693877

In [15]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorIndexer

# Build the Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="indexedFeatures")

# Create the Pipeline
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model
model = pipeline.fit(train)

# Display Decision Tree
display(model.stages[-1])

treeNode
"{""index"":9,""featureType"":""continuous"",""prediction"":null,""threshold"":5.015000000000001,""categories"":null,""feature"":20,""overflow"":false}"
"{""index"":7,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[0.0,1.0],""feature"":13,""overflow"":false}"
"{""index"":1,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[0.0],""feature"":13,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":2.436,""categories"":null,""feature"":20,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":4.5E-4,""categories"":null,""feature"":4,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":8,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [16]:
import numpy as np
import pandas as pd
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer

# Create a random forest pipeline

rf = RandomForestClassifier(labelCol="label", featuresCol="indexedFeatures")
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])
 
# Train model
model = pipeline.fit(train)

# Function to create feature importance dataframe
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

# Store feature importance dataframe
scores = ExtractFeatureImp(model.stages[-2].featureImportances, train, "features")

# Create a table for SQL usage
spark_df = sqlContext.createDataFrame(scores)
spark_df.createOrReplaceTempView("scores")
display(scores)

idx,name,score
20,Inherent_Risk,0.256762540063198
19,Score,0.11645304056542623
21,CONTROL_RISK,0.08156608441129279
12,Risk_D,0.06578918229155964
3,Risk_A,0.06553171506057519
10,Money_Value,0.06080688646399375
6,Risk_B,0.06056284291120133
2,Score_A,0.05940812195361865
1,PARA_A,0.058742304966833546
15,RiSk_E,0.05433943561631396
